<a href="https://colab.research.google.com/github/zHazyl/ml-from-scratch/blob/main/langchain-course/practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
requirements = """
langchain_google_genai
langchain
python-dotenv
huggingface-hub
openai
chromadb
pypdf
"""

with open("requirements.txt", "w") as f:
  f.write(requirements)

In [ ]:
!pip install -r requirements.txt

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
# import google.generativeai as genai
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import MessagesPlaceholder, HumanMessagePromptTemplate, ChatPromptTemplate, PromptTemplate
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory, ConversationSummaryMemory
from dotenv import load_dotenv
import os

from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

chat = ChatGoogleGenerativeAI(
    model="gemini-pro",
    # client=genai,
    temperature=0.3,
    # convert_system_message_to_human=True
)

# chat = HuggingFaceHub(repo_id="abacusai/Liberated-Qwen1.5-72B", model_kwargs={"temperature":0.5, "max_length":512})

# chat = ChatOpenAI()

memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True)

# memory = ConversationSummaryMemory(
#     # chat_memory=FileChatMessageHistory("messages.json"),
#     memory_key="messages",
#     return_messages=True,
#     llm=chat
# )

prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

# memory_prompt_template = """<|im_start|>system
# You are Liberated, a helpful AI assistant.<|im_end|>
# <|im_start|>user
# {content}<|im_end|>
# """

# def create_prompt_from_template(template):
#     return PromptTemplate.from_template(template)

# prompt = create_prompt_from_template(memory_prompt_template)


chain = LLMChain(
    llm=chat,
    prompt=prompt,
    memory=memory
)

while True:

    content = input(">> ")

    result = chain({"content": content})

    print(result['text'])


In [8]:
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import Chroma
from langchain.schema import BaseRetriever

class RedundantFilterRetriever(BaseRetriever):
  embeddings: Embeddings
  chroma: Chroma

  def get_relevant_documents(self, query):
    emb = self.embeddings.embed_query(query)
    return self.chroma.max_marginal_relevance_search_by_vector(
        embedding=emb,
        lambda_mult=0.8
    )

  async def aget_relevant_documents(self):
    return []

In [18]:
from langchain_google_genai import GoogleGenerativeAI
response_model = GoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.2,
    task="text-generation",
    repetition_penalty=1.1,
    return_full_text=True
)

In [19]:
standalone_model = GoogleGenerativeAI(
    model="gemini-pro",
    temperature=0,
    task="text-generation",
    repetition_penalty=1.1,
    return_full_text=True
)

In [13]:
response_model.invoke("System: you are my boss. Me: What's up. You:")

'I need you to work on a project for me.'

In [17]:
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

db = Chroma(
    persist_directory="emb",
    embedding_function=embeddings
)

retriever = RedundantFilterRetriever(
    embeddings=embeddings,
    chroma=db
)

In [21]:
!mkdir pdfs

In [23]:
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load_and_split()

In [24]:
context = "\n".join(str(p.page_content) for p in data)

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [26]:
texts = text_splitter.split_text(context)

In [ ]:
db.add_texts(texts)

In [ ]:
retriever.invoke("What is Spring?")

In [30]:
from langchain.prompts import PromptTemplate

custom_prompt_template = """
### System:
You are an AI assistant that follows instructions extremely well. Help as much as you can.
### User:
You are a research assistant for an artificial intelligence student. Use only the following information to answer user queries:
Context= {context}
History = {history}
Question= {question}
### Assistant:
"""

prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=["question", "context", "history"])

In [31]:
memory = ConversationBufferMemory(input_key="question",
                                   memory_key="history",
                                   return_messages=True)

In [104]:
from langchain_google_genai import ChatGoogleGenerativeAI
response_model = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.2,
    task="text-generation",
    repetition_penalty=1.1,
    return_full_text=True
)

In [52]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

loaded_memory = RunnablePassthrough.assign(
 chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)

In [72]:
from langchain.prompts.prompt import PromptTemplate
from langchain_core.messages import get_buffer_string

_template = """
Given the following conversation and a follow up question,
rephrase the follow up question to be a standalone question, in English,
that can be used to query a FAISS index. This query will be used to retrieve documents with additional context.

Let me share a couple examples.

If you do not see any chat history, you MUST return the "Follow Up Input" as is:
```
Chat History:
Follow Up Input: How is Lawrence doing?
Standalone Question:
How is Lawrence doing?
```

If this is the second question onwards, you should properly rephrase the question like this:
```
Chat History:
Human: How is Lawrence doing?
AI:
Lawrence is injured and out for the season.
Follow Up Input: What was his injury?
Standalone Question:
What was Lawrence's injury?
```

Now, with those examples, here is the actual chat history and input question.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:
[your response here]
"""

STANDALONE_QUESTION_PROMPT = PromptTemplate.from_template(_template)
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | STANDALONE_QUESTION_PROMPT
    | standalone_model,
}

In [105]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
                      llm=response_model,
                      chain_type='map_rerank',
                      retriever = retriever,
                      return_source_documents = True,
                      chain_type_kwargs = {
                          # "verbose": False,
                          # "prompt": prompt,
                          "memory": memory
                          }
                      )

In [106]:
query = "What is Spring?"
response = qa_chain({"query": query})
response['result']

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'Spring is a framework for developing Java applications.'

In [ ]:
query = "What is it purpose?"
response = qa_chain({"query": query})
response['result']

In [111]:
final_chain =  loaded_memory | standalone_question | {"query": itemgetter("standalone_question")} | qa_chain

In [ ]:
inputs = {"question": "What is definition of Reactive Streams?"}
final_chain.invoke(inputs)

In [ ]:
inputs = {"question": "What is it use for"}
final_chain.invoke(inputs)